<a href="https://colab.research.google.com/github/Nikitoniy/Yandex_Data_Analysis/blob/main/5_%D0%A3%D1%80%D0%BE%D0%B2%D0%B5%D0%BD%D1%8C_%D0%BF%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%BE%D0%B9_%D0%BB%D0%BE%D1%8F%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%81%D1%80%D0%B5%D0%B4%D0%B8_%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2_%D0%B8%D0%B7_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Уровень потребительской лояльности среди клиентов из России

**Описание проекта:**

Необходимо определить текущий уровень потребительской лояльности, по-другому NPS, среди клиентов из России. Для этого был проведен опрос и подготовлены таблицы.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники»;
- 7-8 баллов — «нейтралы»;
- 0-6 баллов — «критики».


**Цель проекта:**

В ходе исследования необходимо подготовить данный, соединив SQL таблицы в одну витрину. Итоговую таблицу необходимо преобразовать и подготовить для полной и корректной работы с данными в Tableau. С помощью Tableau нужно дать полный ответ на актуальные вопросы, а также объединить данные в единую презентацию.

**Описание исходных данных (таблиц):**
- Таблица `user` (основная информация о клиентах):
    - `user_id` - идентификатор клиента, первичный ключ таблицы;
    - `lt_day` - количество дней «жизни» клиента;
    - `age`	- возраст клиента в годах;
    - `gender_segment` - пол клиента (1 – женщина, 0 – мужчина);
    - `os_name`	- тип операционной системы;
    - `cpe_type_name` - тип устройства;
    - `location_id`	- идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице `location`;
    - `age_gr_id` - идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице `age_segment`;
    - `tr_gr_id` - идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице `traffic_segment`;
    - `lt_gr_id` - идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице `lifetime_segment`;
    - `nps_score` - оценка клиента в NPS-опросе (от 1 до 10).
- Таблица `location` (территории указанных услуг):
    - `location_id`	- идентификатор записи, первичный ключ;
    - `country`	- страна;
    - `city` - город.
- Таблица `age_segment` (возврастные сегменты клиентов):
    - `age_gr_id`	Идентификатор сегмента, первичный ключ;
    - `bucket_min` - минимальная граница сегмента;
    - `bucket_max` - максимальная граница сегмента;
    - `title` - название сегмента.
- Таблица `traffic_segment` (сегменты потребляемого трафика):
    - `tr_gr_id` - идентификатор сегмента, первичный ключ;
    - `bucket_min` - минимальная граница сегмента;
    - `bucket_max` - максимальная граница сегмента;
    - `title` - название сегмента.
- Таблица `lifetime_segment` (сегменты месяцев жизни клиента):
    - `lt_gr_id` - идентификатор сегмента, первичный ключ;
    - `bucket_min` - минимальная граница сегмента;
    - `bucket_max` - максимальная граница сегмента;
    - `title` - название сегмента.

## Подготовка данных к работе

### Импорт библиотек

In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

### Подключение к базам данных

In [ ]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

### Подготовка таблицы к работе

In [ ]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'Новый'
           WHEN u.lt_day > 365 THEN 'Старый'
       END AS is_new,
       CAST(u.age AS int) AS age,
       CASE
           WHEN u.gender_segment = 1 THEN 'Женщина'
           WHEN u.gender_segment = 0 THEN 'Мужчина'
           ELSE 'Другой'
       END AS gender_segment,
       LOWER(u.os_name) AS os_name,
       LOWER(u.cpe_type_name) AS cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(age.title, 3, length(age.title)-2) AS age_segment,
       SUBSTRING(ts.title, 3, length(ts.title)-2) AS traffic_segment,
       SUBSTRING(ls.title, 3, length(ls.title)-2) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 AND u.nps_score <= 10 THEN 'Сторонники'
           WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN 'Нейтралы'
           WHEN u.nps_score >= 0 AND u.nps_score <= 6 THEN 'Критики'
        END AS nps_group
       FROM user AS u
       LEFT JOIN location AS l ON l.location_id = u.location_id
       LEFT JOIN age_segment AS age ON age.age_gr_id = u.age_gr_id
       LEFT JOIN traffic_segment AS ts ON ts.tr_gr_id = u.tr_gr_id
       LEFT JOIN lifetime_segment AS ls ON ls.lt_gr_id = u.lt_gr_id
       ;
"""

Проверим итоговую таблицу

In [ ]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,android,smartphone,Россия,Уфа,45-54,1-5,36+,10,Сторонники
1,A001WF,2344,Старый,53.0,Мужчина,android,smartphone,Россия,Киров,45-54,1-5,36+,10,Сторонники
2,A003Q7,467,Старый,57.0,Мужчина,android,smartphone,Россия,Москва,55-64,20-25,13-24,10,Сторонники


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Промежуточный итог:**

В ходе подготовки данных была построена едининая таблица с данными, в которую вошли поля:
- user_id;
- lt_day;
- age;
- gender_segment - "1" изменена на "Женщина", "0" - на "Мужчина";
- os_name - приведена к нижнему регистру;
- cpe_type_name - приведена к нижнему регистру;
- country;
- city;
- age_segment - убрано начало строки с порядковым номером группы;
- traffic_segment - убрано начало строки с порядковым номером группы;
- lifetime_segment - убрано начало строки с порядковым номером группы;
- nps_score.

В таблицу добавлены новые колонки:
- is_new - определение нового и старого клиента;
- nps_group - определение группы лояльности пользователя (сторонники, нейтралы, критики) в зависимости от оценок.

## Дашборды в Tableau

https://public.tableau.com/app/profile/nikita.levin/viz/Yandex_project_2_17014578898260/sheet23?publish=yes

**Описание дашбордов:**

1. Обозначение целей и задач исследования.
2. Ознакомление с данными:
    - Карта с наиболее популярными городами по количеству опрошенных пользователей;
    - Гистрограмма распределения пользователей по возрасту;
    - Диаграмма с областями распределения пользователей по возрасту в разрезе их пола;
    - Соотношение новых и старых клиентов.
3. Лояльность клиентов (все графики/таблицы строились с разрезом по группе лояльности):
    - Столбчатая диаграмма по доле пользователей и групп возраста;
    - График лояльности по доле пользователей и трафику;
    - Столбатая диаграмма по доле пользователей и лайфтайма пользователя;
    - Таблица распределения новых и старых клиентов;
    - Таблица распределения по полу пользователей.
4. Уровень NPS среди опрошенных:
    - Соотношение критиков и сторонников;
    - Общий уровень NPS;
    - Столбчатая диаграмма по уровню NPS и используемого ОП в разерезе новых и старых клиентов;
    - Диаграмма 10 популярных городов по уровню NPS в разрезе пола клиентов.
5. Сторонники компании:
    - Соотношение женщин и мужчин сторонников;
    - Соотношение новых и старых клиентов сторонников;
    - 10 популярных городов сторонников;
    - Древовидная карта групп возраста стронников;
    - Таблица с используемыми ОП сторонников;
    - Таблица с устройствами сторонников.
6. Общие выводы по исследованию.

## **Общий вывод по проекту:**

Проект разбит на 3 этапа:

**Первый этап:**

Были подготовлены данные, собранные из нескольких таблиц.

Некоторые колонки были преобразованы:
- gender_segment - "1" изменена на "Женщина", "0" - на "Мужчина";
- os_name - приведена к нижнему регистру;
- cpe_type_name - приведена к нижнему регистру;
- age_segment - убрано начало строки с порядковым номером группы;
- traffic_segment - убрано начало строки с порядковым номером группы;
- lifetime_segment - убрано начало строки с порядковым номером группы.

Также были добавлены новые колонки:
- is_new - определение нового и старого клиента;
- nps_group - определение группы лояльности пользователя (сторонники, нейтралы, критики) в зависимости от оценок.

**Второй этап:**

Были построены дашборды в Tableau, которые отвечают на вопросы:

- Как распределены участники опроса по возрасту и полу?
    - Самый популярный возраст среди клиентов - 34 года, чаще всего возраст клиентов составляет от 27 лет до 41 года;
    - Разница в распределении мужчин и женщин в опросе составляет не более 5%, при этом женщин больше;
- Каких пользователей больше: новых или старых?
    - Старых пользователей больше - 83%, новых - 17%;
- Пользователи из каких городов активнее участвовали в опросе?
    - Чаще всего в опросах принимали участие клиенты в Москве (13% от всех пользователей), Санкт-Петербурге (5.9% от всех пользователей) и Новосибирске (5.3% от всех пользователей);
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
    - Новые клиенты лояльнее;
    - Лояльные клиенты чаще в группах от 35 до 44 лет (17%), самые критикующие клиенты более молодые - от 25 до 34 лет (11%);
    - Разность лояльности клиентов между сторонниками и критиками в разрезе лайфтайма достигает в 2 раза. Больше всего критиков и сторонников в категории 36+ - 18% против 26% соответственно;
    - Чаще всего клиенты являются сторонниками вне зависимости от трафика. Больше всего сторонников и критиков находится в границах потребления трафика 5-10 и составляет 12% и 6% соответственно;
- Какой общий NPS среди всех опрошенных?
    - Общий уровень без каких-либо разрезов составляет 22%;
    - Всего сторонников сервиса 63%, всего критиков сервиса 37% без каких-либо разрезов;
    - Самый высокий общий уровень NPS находится в Иваново - 47%. Самый низкий в Воронеже - 4%. Среди мужчин присутсвует отрицательный уровень NPS, так же в Воронеже - -2%;
    - У новых клиентов в среднем уровень NPS выше, чем у старых. У новых уровень составляет 35%, у старых - 19%;
    - Самый низкий уровень NPS возникает у любых клиентов (старых или новых) с системой "ios". Наивысший уровень у пользователей с системами "bada os" (60% у старых клиентов), "symbain os" (52% у новых клиентов и 30% у старых клиентов), "proprietary" (47% у новых клиентов и 33% у старых клиентов);
    - Стоит отметить, чаще чем старше пользователь, тем выше уровень NPS. Так у пользователей старше 70 уровень NPS выше 50%, а у пользователей 11, 14, 15 лет уровень NPS ниже 0.
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
    - Типичному стороннику от 25 до 44 лет.
    - У данного сторонника с высокой вероятность смартфон (таких пользователей 96%) на ОП "Android" (таких пользователей 88%);
    - Вероятнее такой пользователь будет находиться в Москве;
    - Большинство сторонников являются давнишими (80% от всех сторонников);
    - Вероятнее таким пользователем будет женщина (57%), тем не менее пользователей мужчин составляет (43%).
    
**Третий этап:**    

Рекомендации к повышению уровня анализируемого NPS:
- Стоит обратить внимание на пользователей мужчин, так как женщины больше пользуются сервисом, они чаще являются сторонниками и у них выше уровень NPS;
- Также уровень NPS можно поднять посредством повышения лояльности среди новых клиентов. Новых клиентов всего 17%, новых клиентов сторонников - 19%;
- Такими новыми клиентами могут оказаться молодые пользователи от 16 до 24. Так как такой категории клиентов уделяется меньше внимание. Из них сторонники ~5% пользователей;
- Можно попробовать продолжить развивать другие ниши среди продуктов компании (для телефонов, роутеров и тд). Так как среди сторонников 96% пользуются смартфонами, отсюда можно сделать вывод, что другие категории не являются развитыми;
- При такой доле сторонников среди смартфонов преобладает ОП только «android» - 88%. Если развивать далее услуги для смартфонов, то можно уделить внимание другим ОП.